# Introdução
O objetivo deste trabalho prático é o desenvolver 2 algoritmos de aprendizado de máquina para prever o tempo de permanência de um atleta na NBA, e agrupar os dados dos atletas em grupos.
Foi composto por duas partes: 
Aprendizado supervisionado: Algoritmo KNN (K Nearest Neightbours) e,
Aprendizado não supervisionado: Algoritmo K-means.

In [5]:
import pandas as pd
import numpy as np

## Importando bibliotecas e lendo os dados 

In [6]:
df_treino = pd.read_csv('nba_treino.csv')
df_teste = pd.read_csv('nba_teste.csv')

Separando os atributos (X) e o rótulo (y)

In [7]:
# Separando os atributos (X) e o rótulo (y)
X_treino = df_treino.drop('TARGET_5Yrs', axis=1)
y_treino = df_treino['TARGET_5Yrs']

X_teste = df_teste.drop('TARGET_5Yrs', axis=1)
y_teste = df_teste['TARGET_5Yrs']

# Código

## Funções Auxiliares do KNN

Começamos com uma o cálculo da distância entre 2 pontos, que será usada para sabermos quais são os vizinhos mais próximos.

In [8]:
def distancia_euclidiana(ponto1, ponto2):
    return np.sqrt(np.sum((ponto1 - ponto2) ** 2))

Os vizinhos mais próximos são escolhidos com base na distância euclidiana, e a classe ou valor mais comum entre esses vizinhos é atribuído ao ponto de teste.

In [9]:
def encontrar_vizinhos(X_treino, y_treino, ponto_teste, k):
    distancias = []

    for i in range(len(X_treino)):
        dist = distancia_euclidiana(X_treino.iloc[i], ponto_teste)
        distancias.append((dist, y_treino.iloc[i]))

    distancias = sorted(distancias)[:k]
    vizinhos = [vizinho[1] for vizinho in distancias]

    return vizinhos

Aqui encapsulamos o processo de prever a classe de um ponto de teste. Ele encontra os vizinhos mais próximos e determina a classe predita com base nos rótulos desses vizinhos.

In [10]:
# Função para prever a classe de um ponto de teste
def prever_classe(X_treino, y_treino, ponto_teste, k):
    vizinhos = encontrar_vizinhos(X_treino, y_treino, ponto_teste, k)
    classe_predita = max(set(vizinhos), key=vizinhos.count)
    return classe_predita

Agora treinamos um modelo KNN com um valor específico de k, fazemos previsões nos dados de teste e calculamos várias métricas de desempenho para avaliar o modelo. Esse processo é repetido para diferentes valores de k, conforme a função é chamada com valores diferentes.

In [11]:
# Função para treinar e avaliar o modelo com diferentes valores de k
def avaliar_knn(k):
    y_pred = []

    for i in range(len(X_teste)):
        predicao = prever_classe(X_treino, y_treino, X_teste.iloc[i], k)
        y_pred.append(predicao)

    # Calculando métricas de desempenho
    matriz_confusao = pd.crosstab(np.array(y_teste), np.array(y_pred), rownames=['Real'], colnames=['Predito'])
    acuracia = np.mean(y_teste == y_pred)
    revocacao = matriz_confusao[1][1]/(matriz_confusao[1][1]+matriz_confusao[0][1])
    precisao = matriz_confusao[1][1]/(matriz_confusao[1][1]+matriz_confusao[1][0])
    f1 = (2*precisao*revocacao)/(precisao+revocacao)

    # Exibindo resultados
    print(f'Matriz de Confusão (k={k}):\n{matriz_confusao}')
    print(f'Acurácia: {acuracia:.4f}')
    print(f'Precisão: {precisao:.4f}')
    print(f'Revocação: {revocacao:.4f}')
    print(f'F1: {f1:.4f}\n')

Aqui eu faço os testes.

Gostaria de colocar uma observação para ser considerada na hora da correção.

Meu código tem um resultado bem semelhante ao do SKLearn, como você poderá ver. Porém por algum motivo ele retorna valores diferentes dos que eu testei de outros colegas. Até cheguei a conversar e pedir ajuda, porém não consegui resolver essa questão, mas volto a dizer, acredito que esteja correto, já que mostra a mesma coisa do SKLearn.

In [12]:
# Testando o modelo com diferentes valores de k
valores_k = [2, 10, 17, 50, 75]
for k in valores_k:
    avaliar_knn(k)

Matriz de Confusão (k=2):
Predito   0   1
Real           
0        73  27
1        94  74
Acurácia: 0.5485
Precisão: 0.7327
Revocação: 0.4405
F1: 0.5502

Matriz de Confusão (k=10):
Predito   0    1
Real            
0        58   42
1        57  111
Acurácia: 0.6306
Precisão: 0.7255
Revocação: 0.6607
F1: 0.6916

Matriz de Confusão (k=17):
Predito   0    1
Real            
0        50   50
1        42  126
Acurácia: 0.6567
Precisão: 0.7159
Revocação: 0.7500
F1: 0.7326

Matriz de Confusão (k=50):
Predito   0    1
Real            
0        54   46
1        42  126
Acurácia: 0.6716
Precisão: 0.7326
Revocação: 0.7500
F1: 0.7412

Matriz de Confusão (k=75):
Predito   0    1
Real            
0        54   46
1        42  126
Acurácia: 0.6716
Precisão: 0.7326
Revocação: 0.7500
F1: 0.7412



Podemos ver que aumentar o valor de k parece melhorar o desempenho do modelo, especialmente em termos de acurácia e F1-Score.
Para k=2, a revocação é relativamente baixa, indicando que o modelo tem dificuldade em identificar corretamente as instâncias positivas.
Conforme k aumenta, a precisão se mantém alta e a revocação melhora, sugerindo uma melhor capacidade de balancear a identificação correta de positivos e negativos.

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Função para treinar e avaliar o modelo com diferentes valores de k
def avaliar_knn_SKLearn(k):
    # Criando o modelo KNN
    modelo = KNeighborsClassifier(k)

    # Treinando o modelo
    modelo.fit(X_treino, y_treino)

    # Fazendo previsões no conjunto de validação
    y_pred = modelo.predict(X_teste)

    # Calculando métricas de desempenho
    matriz_confusao = confusion_matrix(y_teste, y_pred)
    acuracia = accuracy_score(y_teste, y_pred)
    precisao = precision_score(y_teste, y_pred)
    revocacao = recall_score(y_teste, y_pred)
    f1 = f1_score(y_teste, y_pred)

    # Exibindo resultados
    print(f'Matriz de Confusão (k={k}):\n{matriz_confusao}')
    print(f'Acurácia: {acuracia:.4f}')
    print(f'Precisão: {precisao:.4f}')
    print(f'Revocação: {revocacao:.4f}')
    print(f'F1: {f1:.4f}\n')


In [14]:
# Testando o modelo com diferentes valores de k
valores_k = [2, 10, 17, 50, 75]
for k in valores_k:
    avaliar_knn_SKLearn(k)

Matriz de Confusão (k=2):
[[73 27]
 [93 75]]
Acurácia: 0.5522
Precisão: 0.7353
Revocação: 0.4464
F1: 0.5556

Matriz de Confusão (k=10):
[[ 58  42]
 [ 57 111]]
Acurácia: 0.6306
Precisão: 0.7255
Revocação: 0.6607
F1: 0.6916

Matriz de Confusão (k=17):
[[ 50  50]
 [ 42 126]]
Acurácia: 0.6567
Precisão: 0.7159
Revocação: 0.7500
F1: 0.7326

Matriz de Confusão (k=50):
[[ 54  46]
 [ 42 126]]
Acurácia: 0.6716
Precisão: 0.7326
Revocação: 0.7500
F1: 0.7412

Matriz de Confusão (k=75):
[[ 54  46]
 [ 42 126]]
Acurácia: 0.6716
Precisão: 0.7326
Revocação: 0.7500
F1: 0.7412



# Discuções

Podemos ver que, meu código parece estar rodando de acordo com o SKLearn, porém com o problema dito mais acima. No mais, tive um pouco mais dificuldade com as métricas e com a função de prever a classe.